In [4]:
import numpy as np
import cv2
import itertools
from scipy.ndimage import gaussian_filter1d
from skimage.filters import sobel

def load_and_preprocess(path):
    try:
        # Standard .img (ENVI/Flat) loading
        img_data = np.fromfile(path, dtype=np.uint8)
        # Assuming 512x512 based on your previous logic
        # If there is a header, we slice the first 512 bytes
        return img_data[512:].reshape((512, 512))
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# ------------------------------------------------------------
# Core Peakiness Threshold
# ------------------------------------------------------------
def threshold_peakiness_advanced(smoothed_hist, min_distance, prominence_thresh):
    """Operates on a pre-smoothed histogram for speed."""
    # Vectorized peak detection
    peaks = np.where(
        (smoothed_hist[1:-1] > smoothed_hist[:-2]) &
        (smoothed_hist[1:-1] > smoothed_hist[2:]) &
        (smoothed_hist[1:-1] > prominence_thresh)
    )[0] + 1

    best_score = -1
    best_valley = 127 # Default mid-point

    for i, j in itertools.combinations(range(len(peaks)), 2):
        p1, p2 = peaks[i], peaks[j]
        if abs(p1 - p2) < min_distance:
            continue

        valley_region = smoothed_hist[p1:p2+1]
        valley_idx = p1 + np.argmin(valley_region)
        valley_height = smoothed_hist[valley_idx]
        
        # Peakiness: Ratio of the shorter peak to the valley floor
        peakiness = min(smoothed_hist[p1], smoothed_hist[p2]) / (valley_height + 1e-6)

        if peakiness > best_score:
            best_score = peakiness
            best_valley = valley_idx

    return best_valley, best_score

# ------------------------------------------------------------
# Optimized Objective Function
# ------------------------------------------------------------
def segmentation_score(image, binary, threshold, peakiness_score, edges, hist):
    """Uses pre-calculated edges and hist to save CPU cycles."""
    
    # 1. Normalized Between-class variance (Otsu)
    # Range scaled to [0, 1]
    prob = hist / hist.sum()
    w0 = prob[:threshold].sum()
    w1 = prob[threshold:].sum()
    
    if w0 == 0 or w1 == 0:
        return -1

    mu0 = np.sum(np.arange(threshold) * prob[:threshold]) / w0
    mu1 = np.sum(np.arange(threshold, 256) * prob[threshold:]) / w1
    bcv = w0 * w1 * (mu0 - mu1) ** 2
    # Empirical normalization for BCV (max possible for 8-bit is ~16384)
    norm_bcv = bcv / 4000 

    # 2. Edge Alignment
    # Measures how well the binary boundary aligns with high-gradient areas
    edge_mask = cv2.dilate(binary, np.ones((3,3))) - cv2.erode(binary, np.ones((3,3)))
    edge_alignment = np.mean(edges[edge_mask > 0]) if np.any(edge_mask) else 0

    # 3. Fast Noise Penalty 
    # Instead of label(), use opening to see how many pixels are 'lost' (noise)
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((3,3)))
    noise_ratio = np.sum(binary != opened) / binary.size

    # Composite score with weighted components
    # Weights: Otsu (40%), Peakiness (30%), Edges (20%), Noise Penalty (10%)
    score = (0.4 * norm_bcv) + (0.3 * peakiness_score) + (0.2 * edge_alignment) - (0.1 * noise_ratio)
    return score

# ------------------------------------------------------------
# Parameter Optimizer
# ------------------------------------------------------------
def optimize_segmentation(image):
    # PRE-CALCULATE static data
    edges = sobel(image)
    hist = np.bincount(image.ravel(), minlength=256).astype(np.float64)
    std_val = np.std(image)

    # Search ranges
    min_dist_range = np.linspace(0.3 * std_val, 0.7 * std_val, 5).astype(int)
    prominence_range = [0.01, 0.05, 0.1]
    smooth_range = [1.0, 2.0, 3.0]

    best_global_score = -np.inf
    best_results = {}

    # Pre-calculate smoothed histograms to avoid repeating in the loop
    smoothed_hists = {s: gaussian_filter1d(hist, sigma=s) for s in smooth_range}

    for smooth_val in smooth_range:
        current_hist = smoothed_hists[smooth_val]
        max_h = current_hist.max()
        
        for min_dist, prom in itertools.product(min_dist_range, prominence_range):
            prom_thresh = max_h * prom
            
            # Find threshold based on peakiness
            T, p_score = threshold_peakiness_advanced(current_hist, min_dist, prom_thresh)
            
            # Generate binary for scoring
            binary = (image >= T).astype(np.uint8) * 255
            
            # Evaluate
            score = segmentation_score(image, binary, T, p_score, edges, hist)

            if score > best_global_score:
                best_global_score = score
                best_results = {
                    "params": (min_dist, prom, smooth_val),
                    "threshold": T,
                    "binary": binary,
                    "score": score
                }

    return best_results

# ------------------------------------------------------------
# Execution
# ------------------------------------------------------------
if __name__ == "__main__":
    img = load_and_preprocess("test1.img")
    
    if img is not None:
        result = optimize_segmentation(img)
        
        print(f"Optimal Threshold: {result['threshold']}")
        print(f"Confidence Score: {result['score']:.4f}")
        print(f"Params (Dist, Prom, Smooth): {result['params']}")
        
        cv2.imwrite("optimized_result.png", result['binary'])

Optimal Threshold: 169
Confidence Score: 1.1527
Params (Dist, Prom, Smooth): (np.int64(20), 0.01, 1.0)
